In [ ]:
'''
A Multilayer Perceptron implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

In [3]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [5]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [6]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [7]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))
    
    # Backup params
    trained_weights = sess.run(weights)
    trained_biases = sess.run(biases)

Epoch: 0001 cost= 158.442778678
Epoch: 0002 cost= 40.613929180
Epoch: 0003 cost= 25.822184559
Epoch: 0004 cost= 17.897572664
Epoch: 0005 cost= 12.819076778
Epoch: 0006 cost= 9.410799195
Epoch: 0007 cost= 7.029883783
Epoch: 0008 cost= 5.139968872
Epoch: 0009 cost= 4.014337767
Epoch: 0010 cost= 2.919351569
Epoch: 0011 cost= 2.162515730
Epoch: 0012 cost= 1.561620276
Epoch: 0013 cost= 1.280861558
Epoch: 0014 cost= 0.965519427
Epoch: 0015 cost= 0.771344704
Optimization Finished!
Accuracy: 0.9448


In [8]:
### Save parameters, inputs and targets into JSON files
import numpy as np
import json

def SimpleEncode(ndarray):
    return json.dumps(ndarray.tolist())
#def SimpleDecode(jsonDump):
#    return np.array(json.loads(jsonDump))

param_names = ["W1", "b1", "W2", "b2", "Wout", "bout"]
params = [trained_weights['h1'].transpose(), trained_biases['b1'],
          trained_weights['h2'].transpose(), trained_biases['b2'],
          trained_weights['out'].transpose(), trained_biases['out']]

i = 0
for param_name in param_names:
    print(str(i) + ". Param \"" + param_name + "\"")
    json_string = SimpleEncode(params[i].astype(np.float32))
    print(params[i].shape)
    #%timeit SimpleDecode(json_string)
    with open(param_name + '.json', 'w') as outfile:
        outfile.write(json_string)
        outfile.close
    print(params[i][0])
    i = i + 1
    
# Test images
json_string = SimpleEncode(mnist.test.images[:100].astype(np.float32))
with open('test_images.json', 'w') as outfile:
    outfile.write(json_string)
    outfile.close
# Test labels
json_string = SimpleEncode(mnist.test.labels[:100].astype(np.float32))
with open('test_labels.json', 'w') as outfile:
    outfile.write(json_string)
    outfile.close
#7.12256670e-01

0. Param "W1"
(256, 784)
[  1.26700139e+00  -4.43212658e-01   6.09928966e-01  -8.01519334e-01
   1.01956987e+00   7.54097462e-01   7.20799923e-01  -4.41826105e-01
   3.08871448e-01   2.26510823e-01   4.19632226e-01  -1.22040339e-01
  -1.97065139e+00   3.34972888e-01   1.63647190e-01   2.54302323e-01
   2.84723341e-01   6.30752385e-01   3.88494730e-01   6.61761284e-01
  -1.89316034e+00   1.62547350e+00   2.27689631e-02  -2.48122141e-01
  -7.87550092e-01   2.54230469e-01  -3.10872551e-02   4.58284795e-01
   5.86339653e-01  -2.06867695e-01  -1.08969879e+00   1.64718223e+00
  -1.95504740e-01  -1.42129228e-01  -5.54778337e-01   4.06386778e-02
   1.57056856e+00  -3.81438583e-01  -1.82694852e-01   2.82075644e-01
   1.56796181e+00   2.68388212e-01   2.34907174e+00  -2.70374883e-02
   2.84780622e-01  -4.22200412e-01   1.89768600e+00   4.48873162e-01
   1.40869927e+00   9.71986949e-01   1.99938107e+00  -5.90975471e-02
  -8.22637022e-01  -4.97181237e-01  -9.37261939e-01   7.65104771e-01
  -1.3940

In [22]:
### Save Tensorflow's forward propagation results into JSON ifle
inputs_total = 10000
x = tf.placeholder("float", [None, n_input])

# Construct model
pred = multilayer_perceptron(x, trained_weights, trained_biases)
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Produce outputs
    result = sess.run([pred], feed_dict={x: mnist.test.images[:inputs_total]})
    
# Print results
print("Output[:2]:")
print(result[0][:2])
print("Output[:2] maxed:")
print([list(decision).index(max(decision)) for decision in result[0][:2]])
print("Correct[:2]:")
print([list(decision).index(max(decision)) for decision in mnist.test.labels[0:2]])
    
# Backup results
json_string = SimpleEncode(result[0].astype(np.float32))
with open('test_tf_results.json', 'w') as outfile:
    outfile.write(json_string)
    outfile.close

Output[:2]:
[[ -237.15965271   642.08825684   594.06115723   653.9631958    -13.95187664
    333.62982178  -856.8449707   1259.04553223   536.81286621
    784.20300293]
 [ -120.90952301   277.98376465  1602.71386719   586.84735107
   -680.02850342   254.78291321   188.31906128  -226.9961853    737.79748535
   -544.57794189]]
Output[:2] maxed:
[7, 2]
Correct[:2]:
[7, 2]
